In [41]:
from tensorflow.keras.datasets import cifar10, mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [42]:
import plotly.express as px

def plot_mnist_image(image):
    fig = px.imshow(image, color_continuous_scale=px.colors.sequential.gray_r)
    fig.show()
    
def plot_rel(image):
    fig = px.imshow(image, color_continuous_scale=px.colors.sequential.Cividis)
    fig.show()

#color_continuous_scale=px.colors.sequential.Cividis_r

In [43]:
def get_weights(model: tf.keras.Sequential) -> (np.ndarray, np.ndarray):
    #TODO return array for variable number of layers

    first_weights = model.weights[0].numpy()
    second_weights = model.weights[1].numpy()

    return first_weights, second_weights


def calc_r(r: np.ndarray, output: np.ndarray, weights: np.ndarray, eps: int = 0, beta: int = None):

    nominator = np.multiply(np.transpose(output),
                            weights)

    
    denominator = np.matmul(output,
                            weights)

    fraction = np.divide(nominator, denominator)

    r_new = np.dot(fraction, r)

    return r_new


# Funktion für Relevance Propagation
def rel_prop(model: tf.keras.Sequential, input: np.ndarray, eps: float = 0, beta: float = None) -> np.ndarray:
    first_weights, second_weights = get_weights(model)

    # Hilfsmodel zum Extrahieren der Outputs des Hidden Layers
    extractor = tf.keras.Model(inputs=model.inputs,
                               outputs=[layer.output for layer in model.layers])

    features = extractor(np.array([input]))

    flattened_input = features[0].numpy()
    hidden_output = features[1].numpy()
    output = features[2].numpy()

    # Berechnung von R1
    r2 = np.transpose(output)

    r1 = calc_r(r=r2,
                output=hidden_output,
                weights=second_weights,
                eps=eps,
                beta=beta)

    r0 = calc_r(r=r1,
                output=flattened_input,
                weights=first_weights,
                eps=eps,
                beta=beta)

    relevance = np.reshape(r0, input.shape)

    return relevance

In [49]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
#from tensorflow.keras.layers import ReLU
from tensorflow.keras.optimizers import Adam
#from sklearn.preprocessing import StandardScaler
#from tensorflow.keras.activations import softmax, relu
from tensorflow.keras.datasets import cifar10, mnist
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

class binary_classifier:
    
    def __init__(self, model_type, data_set, class_nb):
        self.model_type = model_type
        assert(type(self.model_type)==str)
        self.data_set = data_set
        self.class_nb = class_nb
        

    def set_data(self, data):
        train_images = data[0]
        train_labels =  data[1]
        self.test_images = data[2]
        self.test_labels = data[3]
        
        #make_binary_data
        train_labels = (train_labels==self.class_nb).astype(int)
        self.test_labels = (self.test_labels==self.class_nb).astype(int)
        
         # reduce train dataset
        one_indices = [i for i in range(train_labels.shape[0]) if train_labels[i]==1]
        zero_indices = [i for i in range(train_labels.shape[0]) if train_labels[i]==0]
        sampling = random.choices(zero_indices, k=3*len(one_indices))
        train_indices = one_indices + sampling
        print("Number of train indices: ", len(train_indices))
        self.train_images = np.asarray([train_images[i] for i in train_indices])
        print(self.train_images.shape)
        self.train_labels = np.asarray([train_labels[i] for i in train_indices])
        
        
        

    def set_model(self):
        
        if self.data_set == 'mnist':
            input_shape=(28,28,1)
        else:
            input_shape=(32,32,3)
        
        if self.model_type == "dense":
            model = Sequential([
                Flatten(input_shape=input_shape),
                Dense(4096, activation='relu', use_bias=False),
                Dense(1, activation='sigmoid', use_bias=False)
            ])

        model.summary()

        model.compile(loss='binary_crossentropy',
                    optimizer=Adam(),
                    metrics=['acc'])

        self.model = model


    def fit_model(self, epochs: int, batch_size: int):
        with tf.device("/gpu:0"):
            self.model.fit(
                self.train_images,
                self.train_labels,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(self.test_images, self.test_labels)
            )

    def predict(self, image):
        pred = self.model.predict(np.array([image]))
        return pred
    
    def non_trivial_accuracy(self):
        answers = []
        for i in range(len(list(self.test_labels))):
            if self.test_labels[i]==1:
                answers.append(int(self.model.predict(np.array([self.test_images[i]]))[0][0]))
                
        return sum(answers)/len(answers)
    
    def evaluate(self, batch_size):
        _ , acc = self.model.evaluate(self.test_images, self.test_labels,
                                batch_size=batch_size)
        return acc

In [50]:
from tensorflow.keras.datasets import cifar10, mnist
import tensorflow as tf
    
def run_binary(dataset, model_type, class_nb):
    if dataset == 'mnist':
        (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    elif dataset == 'cifar10':
        (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
        
    data = [train_images, train_labels, test_images, test_labels]
    
    cl = binary_classifier(model_type="dense", data_set=dataset, class_nb=class_nb)
    cl.set_data(data)
    cl.set_model()
    epochs = 10
    batch_size = 100
    cl.fit_model(epochs, batch_size)

    print("Model Accuracy: {}".format(cl.evaluate(10)))
    print("Model Accuracy for images with label {} : {}".format(class_nb, cl.non_trivial_accuracy()))
    
    model = cl.model
    #model.save('./binary_models/model_{}_{}_{}e_{}bs.h5'.format(dataset,model_type,epochs,batch_size))
    
    return model
    
    

In [51]:
model = run_binary(dataset='mnist', model_type='dense', class_nb=5)

Number of train indices:  21684
(21684, 28, 28)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4096)              3211264   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 4096      
Total params: 3,215,360
Trainable params: 3,215,360
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
217/217 [==============================] - 1s 3ms/step - loss: 9.7641 - acc: 0.9466 - val_loss: 0.1137 - val_acc: 0.9834
Epoch 2/10
217/217 [==============================] - 1s 3ms/step - loss: 0.0605 - acc: 0.9898 - val_loss: 0.0647 - val_acc: 0.9919
Epoch 3/10
217/217 [==============================] - 1s 3ms/s

In [55]:
j=0
i=0
while j<10:
    if test_labels[i]==5:
        j+=1
        image = test_images[i]
        prop = rel_prop(model, image)
        plot_rel(prop)
    i+=1

In [ ]:
relevances = []
    for i in range(100):
        plot_rel_prop(model,test_images[i], eps=None, beta=None)
        relevances.append(rel_prop(model, test_images[i]))
    
    for i in range(len(relevances)):
        print(test_labels[i], (relevances[i]!=0).any())
        
    return relevances
    